In [0]:
%load_ext Cython

In [0]:
%%cython
# encoding: utf8
from __future__ import unicode_literals
import numpy as np
import random
import matplotlib.mlab as mlab

cdef extern from "math.h":
    double exp(double)
    double sqrt(double)
    double log(double)

cdef class MultiOutputGP:
    cdef double beta
    cdef int N, D
    cdef xt, yt
    cdef double[:,:] i_cov
    cdef double[:] param

    # 入力の相関を計算するカーネル
    cdef double k_xx(self, xi, xj):
        cdef double theta0 = 1.0
        cdef double theta1 = 1.0
        cdef double theta2 = 0
        cdef double theta3 = 16.0
        return theta0 * exp(-0.5 * theta1 * np.sum((xi - xj) * (xi - xj))) + theta2 + theta3 * np.sum(xi * xj)

    # 入力の各次元の相関を計算するカーネル
    cdef double k_dd(self, d1, d2):
        if d1==d2:
          return 1.0
        else:
          return 0.5

    def __init__( self ):
        self.beta = 10
        self.param_cache = {}

    def learn(self, xt, yt ):
        cdef int i,j,d1,d2
        self.xt = xt
        self.yt = yt
        self.N = len(xt)
        self.D = len(xt[0])
        cdef double[:,:] cov = np.zeros((self.N*self.D, self.N*self.D))

        for d1 in range(self.D):
            for d2 in range(self.D):
                for i in range(self.N):
                    for j in range(self.N):
                        ii = d1*self.N+i
                        jj = d2*self.N+j

                        cov[ii, jj] = self.k_xx(xt[i], xt[j]) * self.k_dd(d1, d2)
                        #if i==j:
                            #cov[ii, jj] += 1/self.beta
                            #cov[ii,jj] += 1.0

                        if ii==jj:
                            cov[ii, jj] += 1/self.beta

        self.i_cov = np.linalg.inv(cov)

        tt = yt.T.flatten()
        self.param = np.dot(self.i_cov, tt)

    def predict( self, x ):
        mus = []
        sigmas = []
        M = len(x)

        for m in range(M):
            v = np.zeros((self.D, self.N*self.D))
            c = np.zeros((self.D, self.D))
            for d1 in range(self.D):
              for d2 in range(self.D):
                for n in range(self.N):
                    v[d1, d2*self.N+n] = self.k_xx(x[m], self.xt[n]) * self.k_dd(d1, d2)
                c[d1, d2] = self.k_xx(x[m], x[m]) * self.k_dd(d1, d2) + 1.0 / self.beta
            
            a = np.array(self.param)
            mu = np.dot(v, self.param)
            sigma = c - np.dot(v, np.dot(self.i_cov, v.T))
            mus.append(mu)
            sigmas.append(sigma)
        
        return np.array(mus), np.array(sigmas)


In [0]:
gp = MultiOutputGP()

In [0]:
xt = np.array([[1,1], [2,2], [3,3], [4,4]])
yt = np.array( [[1,2], [3,4], [2,1], [6,8]] )

In [0]:
gp.learn(xt, yt)

In [310]:
mu, sig = gp.predict( np.array([[2,2]]) )

print(mu)
print(sig)

[[2.94232979 3.76924004]]
[[[0.18644903 0.10576748]
  [0.10576748 0.18644903]]]
